In [82]:
from langchain_community.document_loaders import UnstructuredURLLoader

In [ ]:
urls = ['https://www.victoriaonmove.com.au/local-removalists.html','https://victoriaonmove.com.au/index.html','https://victoriaonmove.com.au/contact.html']
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [84]:
data

[Document(metadata={'source': 'https://shortlink37.vercel.app'}, page_content="Transform Your Links, Elevate Your Reach\n\nCreate short, memorable links in seconds. Track their performance and ensure safety with our advanced URL shortening platform.\n\nCustom Short Links\n\nCreate branded, memorable URLs\n\nAI-Powered Safety\n\nAutomatic URL safety verification\n\nDetailed Analytics\n\nTrack and analyze link performance\n\nHow It Works\n\n1. Paste Your URL\n\nEnter your long URL and optionally customize your short link\n\n2. AI Safety Check\n\nOur AI automatically verifies the safety of your URL\n\n3. Track & Analyze\n\nMonitor your link's performance with detailed analytics\n\nPowerful Features\n\nCustom Domains\n\nUse your own domain for branded short links that reinforce your identity\n\nAdvanced Controls\n\nSet expiration dates, password protection, and geographic restrictions\n\nOur Impact\n\n20+\n\nLinks Created\n\n5+\n\nActive Users\n\n4+\n\nTotal Clicks\n\nGet Started FreeView 

In [85]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  1


In [105]:
docs[0]

Document(metadata={'source': 'https://shortlink37.vercel.app'}, page_content="Transform Your Links, Elevate Your Reach\n\nCreate short, memorable links in seconds. Track their performance and ensure safety with our advanced URL shortening platform.\n\nCustom Short Links\n\nCreate branded, memorable URLs\n\nAI-Powered Safety\n\nAutomatic URL safety verification\n\nDetailed Analytics\n\nTrack and analyze link performance\n\nHow It Works\n\n1. Paste Your URL\n\nEnter your long URL and optionally customize your short link\n\n2. AI Safety Check\n\nOur AI automatically verifies the safety of your URL\n\n3. Track & Analyze\n\nMonitor your link's performance with detailed analytics\n\nPowerful Features\n\nCustom Domains\n\nUse your own domain for branded short links that reinforce your identity\n\nAdvanced Controls\n\nSet expiration dates, password protection, and geographic restrictions\n\nOur Impact\n\n20+\n\nLinks Created\n\n5+\n\nActive Users\n\n4+\n\nTotal Clicks\n\nGet Started FreeView P

In [87]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [88]:
from dotenv import load_dotenv
load_dotenv()

True

In [89]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [90]:
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [101]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [102]:
retrieved_docs = retriever.invoke("What kind of services they provide?")

In [103]:
len(retrieved_docs)

3

In [104]:
retrieved_docs

[Document(id='eb6448b8-da4a-474e-9e1e-d8d5b791378a', metadata={'source': 'https://victoriaonmove.com.au/index.html'}, page_content="2 BED HOME:\n\n2 Beds, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes, Loose stuff.\n\nBig HOME:\n\n4-5 Bed Room, Double Lounge, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes,Furniture, Loose stuff. Get a Quote\n\nInterstate Removalists:\n\nSeamless Relocation Across State Lines\n\n\n\nMelbourne to Sydney Removalists\n\nBook Now\n\n\n\nMelbourne to Brisbane Removalists\n\nBook Now\n\n\n\nMelbourne to Adelaide Removalists\n\nBook Now\n\n\n\nMelbourne to Canberra Removalists\n\nBook Now\n\n\n\nOUR MOVING WORK & ETHICS\n\nMoving house furniture is an art that comes with experience and a steadfast commitment to our customers' needs. It's a challenging industry, and not everyone survives for long without the expertise and dedication that we bring to every job. Our long history 

In [95]:
print(retrieved_docs[0].page_content)

2 BED HOME:

2 Beds, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes, Loose stuff.

Big HOME:

4-5 Bed Room, Double Lounge, Mattress, Tall Boy, Bedside, Refrigerator, Washing Machine,Dresser, Barbeque, Some Boxes,Furniture, Loose stuff. Get a Quote

Interstate Removalists:

Seamless Relocation Across State Lines



Melbourne to Sydney Removalists

Book Now



Melbourne to Brisbane Removalists

Book Now



Melbourne to Adelaide Removalists

Book Now



Melbourne to Canberra Removalists

Book Now



OUR MOVING WORK & ETHICS

Moving house furniture is an art that comes with experience and a steadfast commitment to our customers' needs. It's a challenging industry, and not everyone survives for long without the expertise and dedication that we bring to every job. Our long history as Removalists in Melbourne is a testament to our sincerity and hard work.


In [96]:
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-2.0-flash")

In [97]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [98]:

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [99]:
response = rag_chain.invoke({"input": "What kind of services they provide?"})
print(response["answer"])

They provide removalist services for 2-bed homes, big homes, and interstate relocations. They offer interstate removalist services from Melbourne to Sydney, Brisbane, Adelaide, and Canberra. Their moving work and ethics demonstrate a commitment to customers' needs with expertise and dedication.
